### 目標－求假設C值下的母體標準差
* 透過Black-scholes公式 (參數: S、K、r、T) 以及樣本標準差，計算結果得報酬率 C 
* 不斷嘗試帶入>0 的浮點數至樣本標準差計算 (利用二分法計算可大幅減少雲算時間)
* 直到計算結果 C 與假設的C值相等，此時帶入的 樣本標準差值 = 假設C值下的母體標準差

### 公式－Black-scholes formula
* C = SN*(d1) - ke ** (-rT)次方 * N(d2)
* 其中 d1 = [ ln(S/K) + (r + σ/2) T ] / σ√T ， d2 = d1 - σ√T
* N(z) 為標準常態分配之累積累積分配函數 (cdf)

### 變數設定
* 參數 S、K、r、T = S、K、r、T
* 公式 d1、d2、N(d1)、N(d2) = d1、d2、N_d1、N_d2
* 樣本標準差： SD_sample
* 計算結果市價： C_sample
* 母體標準差： SD_population
* 假設市價C值： C_population

In [1]:
# 導入模組

import time
import random
import math
from scipy.stats import norm   # 計算cdf norm.cdf(z,loc = 0 ~平均值,scale = 1~標準差) 求cdf


In [2]:
# 公式撰寫

def Black_scholes(SD_sample) :
    
    S = 1
    K = 1
    r = 0.1
    T = 1
    
    d1 = (math.log(S/K) + (r + pow(SD_sample,2)/2)*T) / SD_sample*pow(T,0.5)
    N_d1 = norm.cdf(d1)
    d2 = d1 - SD_sample*pow(T,0.5)
    N_d2 = norm.cdf(d2)
    C_sample = S*N_d1 - K*math.exp(-r*T)*N_d2
    
    return C_sample


In [3]:
N = 10              # 帶入的標準差的最大值
a = 1000000         # 0~N 中隨機抽取約 a 個浮點數做運算
error = 0.0004       # C 值間的容許誤差
C_max = Black_scholes(N)

C_population = C_max+1
while C_population > C_max :
    print('請輸入 0 到',C_max,'之間的數')
    C_population = float(input('假設市價 C 值：'))



請輸入 0 到 0.9999994546826071 之間的數
假設市價 C 值：0.5


In [4]:
# 建立 SD_sampledate 的 list，資料為 0~N 間不重複的樣本標準差 SD_sample
def SD_sampledate(a) :
    
    date_set = set()                           # 設計date_set，集合中約有 a 個 0~N 間不重複的浮點數
    for i in range(a) :
        date_set.add(random.uniform(0, N))

    date_list = list(date_set)                 # 將集合轉換成清單list_index
    date_list.sort()                           # 排序 (其實可以不用此步驟，已是排序好的)
    return date_list



#建立 SD_samplelist 清單，資料為 0~N 間不重複的樣本標準差 SD_sample
SD_samplelist = SD_sampledate(a)



In [5]:
# 二分法

list_ans = SD_samplelist[:]          # 輸入二分法需要的變數
b = C_population

def number_index(b,list_ans) :
    
    start = time.time()              # 時間計算
    print('起始時間：',start)
    
    count = 0
    while b - Black_scholes(list_ans[0]) > error :   # 確認 b 是否和清單中第一個值的運算結果相等，如果若不是則重複做分割清單的動作
        len_left = round(len(list_ans)/2+0.1)        # 找出list一半的索引值，若list共有奇數筆資料，將前半部資料設為多數
        list_left = list_ans[0:len_left]             # 將list_ans分割成list_left和list_right兩個清單
        list_right = list_ans[len_left:]
        
        if b < Black_scholes(list_right[0]) :        # 判斷 b 是否在前半部分list_left中
            list_ans = list_left[:]                  # 若 "是" ，將list_left回傳
        elif b >= Black_scholes(list_right[0]) :     # 判斷 b 是否在後半部分list_right中
            list_ans = list_right[:]                 # 若 "是" ，將list_right回傳
            count = count + len_left                 # 並累加list_right[0]在原本list_ans中的索引值，即count
    
    number_index = count                             # 離開迴圈，即 b 為回傳list_ans[0]，list_ans.index(b)為累加的count
    
    end = time.time()                 # 時間計算
    print('結束時間：',end)
    print('執行時間：',end - start )
    
    return number_index                              # 回傳目標在 SD_samplelist 的索引值

SD_index = number_index(b,list_ans)
SD_population = SD_samplelist[SD_index]
print('假設市價為',C_population,'的情況下，標準差為',SD_population)

起始時間： 1677158975.499702
結束時間： 1677158975.7310836
執行時間： 0.23138165473937988
假設市價為 0.5 的情況下，標準差為 1.267153898837301
